In [4]:
import pandas as pd
hyka=pd.read_csv(r"D:\项目\大冶\陈贺发送数据\文件\下线会员.csv",encoding="ANSI")
hyka.iloc[:,:]=hyka.iloc[:,:].astype("string")
hyka.to_excel(r'D:\项目\大冶\需要数据\下线会员.xlsx',index=False)

In [5]:
#所有层级代理关系
import pandas as pd
from graphviz import Digraph
import numpy as np
def find_allxiaji_proxies(df, proxyName, result=[]):
    result= []
    """
    Recursive function to find all proxies for a given proxyName in a pandas DataFrame.
    """
    # Find all rows where proxyName is in the proxyName column
    rows = df[df['proxyName'] == proxyName]
    # If there are no rows, return the result
    if rows.empty:
        return result
    # Otherwise, loop through each row and add the username to the result list
    for index, row in rows.iterrows():
        result.append(row['username'])
        # Recursively call the function with the new proxyName
        find_allxiaji_proxies(df, row['username'], result)
    return result
def find_allshangji_proxies(df, username, result=[]):
    result= []
    """
    Recursive function to find all proxies for a given proxyName in a pandas DataFrame.
    """
    # Find all rows where proxyName is in the proxyName column
    rows = df[df['username'] == username]
    # If there are no rows, return the result
    if rows.empty:
        return result
    # Otherwise, loop through each row and add the username to the result list
    for index, row in rows.iterrows():
        result.append(row['proxyName'])
        # Recursively call the function with the new proxyName
        find_allshangji_proxies(df, row['proxyName'], result)
    return result
# def df_to_dict(df):
#     """
#     将三列数据的DataFrame转换为字典
#     """
#     result = df.set_index(df.columns[0]).to_dict('index')
#     return result
def df_to_dict(df):
        result = {}
        for _, row in df.iterrows():
            account = row['账号']
            branch = row['银行卡开户支行']
            card_number = row['提取银行卡号']

            if account not in result:
                result[account] = []

            result[account].append((branch, card_number))
        return result
df = pd.read_excel(r'D:\项目\大冶\需要数据\下线会员.xlsx', sheet_name=0,dtype={'proxyName':str,'username':str})
df_yh = pd.read_excel(r"D:\项目\大冶\需要数据\会员银行卡.xlsx", sheet_name="Sheet4",dtype={'提取银行卡号':str})
hy_jz= pd.read_excel(r"D:\项目\大冶\需要数据\会员银行卡.xlsx", sheet_name="Sheet3",dtype={'提取银行卡号':str})

# hy_ka1=hy_jz[['USER_NAME','PAY_ACCOUNT']]
# hy_ka=hy_ka1.rename(columns={'USER_NAME':'查询账号','PAY_ACCOUNT':'账户'})
result_yh_dt = df_to_dict(df_yh)
agent_find = pd.read_excel(r"D:\项目\大冶\需要找的代理.xlsx", sheet_name=0)
agent_l=list(agent_find['账号'])
l_all_ka=[]
zh=[]
kas=[]
for i in agent_l:
    result_xia = find_allxiaji_proxies(df,i)
    #print(result_xia)
    result_shang = find_allshangji_proxies(df,i)
    data1=df[df['username'].isin(result_xia)]
    data2=df[df['proxyName'].isin(result_shang)]
    data=pd.concat([data1,data2])
    #print(data)
    fi_data = data[(data['proxyName'] != 'dmg008') | (data['username'] ==i)]
    #print(fi_data)
    l_all_ka.append(fi_data)
    photo = Digraph('%s' % (i), format='pdf')  # 创建photo图像
    photo.graph_attr['rankdir'] = 'LR' 
    nodes = {}  # 存储所有节点名称的字典

    for index, row in fi_data.iterrows():
            mother = row['proxyName']  # 母公司名称

            if mother in result_yh_dt :
                    #mother_name = str(mother)+ '\n'+str(result_yh_dt[mother]['收款账号'])+'\n'+str(result_yh_dt[mother]['身份证'])
                    mother_name = str(mother)+ '\n'+str(result_yh_dt[mother])
                    #print(mother_name)
            else:
                    mother_name = str(mother)
            child = row['username']
            if child in result_yh_dt :
                #child_name = str(child)+ '\n'+str(result_yh_dt[child]['收款账号'])+'\n'+str(result_yh_dt[child]['身份证'])
                    child_name = str(child)+ '\n'+str(result_yh_dt[child])
            else:
                    child_name = str(child)
    #         else:
    #             s=s+row['用户名 姓名 登录IP']+'\n'
            if mother_name not in nodes.keys():  # 如果没有画过该节点
                photo.node(name=str(mother_name), fontname="Microsoft YaHei")  # 画母公司节点

            if child_name not in nodes.keys():
            # print(str(index))
                photo.node(name=str(child_name), fontname="Microsoft YaHei")  # 画子公司节点

                photo.edge(str(child_name),str(mother_name))
            # print(nodes.items())
            # photo.write_svg(r'D:\代理图\8888.png', prog = 'dot')
    photo.render(filename='%s' % (i),outfile= None,directory=r'D:\项目\大冶\下线会员\{}'.format(i))
    u1=list(fi_data['username'])
    p1=list(fi_data['proxyName'])
    l=list(set(u1+p1))
    data_info=pd.DataFrame({'账号':l})
    data_info['所在代理']=i

    
    data_ka=data_info.merge(df_yh,on='账号',how='left')
    zh.append(data_ka)
    agent_info = data_ka[pd.notnull(data_ka['银行卡开户支行']) & data_ka['银行卡开户支行'].str.contains('湖北省黄石')]
    agent_info['所在代理']=i

    kas.append(agent_info)
    fi_data=fi_data.rename(columns={'proxyName':'所属账户','username':'无方框账户'})
    with pd.ExcelWriter(r'D:\项目\大冶\下线会员\{}\{}.xlsx'.format(i, i)) as writer:
            fi_data.to_excel(writer, sheet_name="关系表",index=False)
            data_ka.to_excel(writer, sheet_name="账号匹配",index=False)
            agent_info.to_excel(writer, sheet_name="黄石账号信息",index=False)
    #data.to_excel(r'C:\Users\23934\Desktop\\代理图1\{}\{}.xlsx'.format(i, i),index=False)
#l_all_ka.to_excel(r'D:\项目\鄂州\代理图\银行卡信息.xlsx',index=False)
agent_kas=pd.concat(l_all_ka)
agent_kas.to_excel(r'D:\项目\大冶\下线会员\关系信息.xlsx',index=False)
zhs=pd.concat(zh)
zhs=zhs.drop_duplicates()
zhs.to_excel(r'D:\项目\大冶\下线会员\银行卡信息.xlsx',index=False)
ka=pd.concat(kas)
ka=ka.drop_duplicates()
ka.to_excel(r'D:\项目\大冶\下线会员\黄石银行卡信息.xlsx',index=False)

C:\Users\23934\AppData\Local\Temp\ipykernel_25540\3155837861.py:120: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  agent_info['所在代理']=i
C:\Users\23934\AppData\Local\Temp\ipykernel_25540\3155837861.py:120: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  agent_info['所在代理']=i
C:\Users\23934\AppData\Local\Temp\ipykernel_25540\3155837861.py:120: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the

In [6]:
import pandas as pd
df = pd.read_excel(r'D:\项目\大冶\需要数据\下线会员.xlsx',sheet_name=0,dtype={'proxyName':str,'username':str})

data = df.groupby('proxyName')['username'].apply(list).to_dict()
# 递归函数，查找指定用户的下属
def find_subordinates(user, target_level, level=0):
    subordinates = []
    if user in data and level < target_level:
        for subordinate in data[user]:
            subordinates.append((level + 1, subordinate, '上级'))
            subordinates += find_subordinates(subordinate, target_level, level=level+1)
    return subordinates

# 递归函数，查找指定用户的上级
def find_superiors(user, target_level, level=0):
    superiors = []
    for k, v in data.items():
        if user in v:
            superiors.append((level + 1, k, '下级'))
            if level < target_level:
                superiors += find_superiors(k, target_level, level=level+1)
    return superiors

# 查找al2162645的下属和上级，限制最大等级为2
agent_find = pd.read_excel(r"D:\项目\大冶\需要找的代理.xlsx", sheet_name=0)
agent_l=list(agent_find['账号'])
ag_s1_x3=[]
for i in agent_l:
    subordinates = find_subordinates(i, 3)
    superiors = find_superiors(i, 3)

    # 将下属和上级的结果合并，并转换成DataFrame并按照等级和用户名排序
    result = sorted(subordinates + superiors, key=lambda x: (x[0], x[1]))
    df_result = pd.DataFrame(result, columns=['level', 'username', 'type'])
    df_result['账户']=i
    ag_s1_x3.append(df_result)
ag_s1_x3s=pd.concat(ag_s1_x3)
ag_s1_x3s=ag_s1_x3s.drop_duplicates()
ag_s1_x3s.to_excel(r'D:\项目\大冶\需要数据\下线会员ag_s1_x3信息.xlsx',index=False)
# 输出结果
#print(df_result)

# # 输出指定用户的下属和上级数量
# num_subordinates = len(subordinates)
# num_superiors = len(superiors)
# print(f"{num_subordinates}个下属，{num_superiors}个上级。")

In [7]:
#所有层级代理关系
import pandas as pd
from graphviz import Digraph
import numpy as np
def find_allxiaji_proxies(df, proxyName, result=[]):
    result= []
    """
    Recursive function to find all proxies for a given proxyName in a pandas DataFrame.
    """
    # Find all rows where proxyName is in the proxyName column
    rows = df[df['proxyName'] == proxyName]
    # If there are no rows, return the result
    if rows.empty:
        return result
    # Otherwise, loop through each row and add the username to the result list
    for index, row in rows.iterrows():
        result.append(row['username'])
        # Recursively call the function with the new proxyName
        find_allxiaji_proxies(df, row['username'], result)
    return result
def find_allshangji_proxies(df, username, result=[]):
    result= []
    """
    Recursive function to find all proxies for a given proxyName in a pandas DataFrame.
    """
    # Find all rows where proxyName is in the proxyName column
    rows = df[df['username'] == username]
    # If there are no rows, return the result
    if rows.empty:
        return result
    # Otherwise, loop through each row and add the username to the result list
    for index, row in rows.iterrows():
        result.append(row['proxyName'])
        # Recursively call the function with the new proxyName
        find_allshangji_proxies(df, row['proxyName'], result)
    return result
# def df_to_dict(df):
#     """
#     将三列数据的DataFrame转换为字典
#     """
#     result = df.set_index(df.columns[0]).to_dict('index')
#     return result
def df_to_dict(df):
        result = {}
        for _, row in df.iterrows():
            account = row['账号']
            branch = row['银行卡开户支行']
            card_number = row['提取银行卡号']

            if account not in result:
                result[account] = []

            result[account].append((branch, card_number))
        return result
df = pd.read_excel(r'D:\项目\大冶\需要数据\下线会员.xlsx', sheet_name=1,dtype={'proxyName':str,'username':str})
df_yh = pd.read_excel(r"D:\项目\大冶\需要数据\会员银行卡.xlsx", sheet_name="Sheet4",dtype={'提取银行卡号':str})
hy_jz= pd.read_excel(r"D:\项目\大冶\需要数据\会员银行卡.xlsx", sheet_name="Sheet3",dtype={'提取银行卡号':str})

# hy_ka1=hy_jz[['USER_NAME','PAY_ACCOUNT']]
# hy_ka=hy_ka1.rename(columns={'USER_NAME':'查询账号','PAY_ACCOUNT':'账户'})
result_yh_dt = df_to_dict(df_yh)
agent_find = pd.read_excel(r"D:\项目\大冶\需要找的代理.xlsx", sheet_name=0)
agent_l=list(agent_find['账号'])
l_all_ka=[]
zh=[]
kas=[]
for i in agent_l:
    result_xia = find_allxiaji_proxies(df,i)
    #print(result_xia)
    result_shang = find_allshangji_proxies(df,i)
    data1=df[df['username'].isin(result_xia)]
    data2=df[df['proxyName'].isin(result_shang)]
    data=pd.concat([data1,data2])
    #print(data)
    fi_data = data[(data['proxyName'] != 'dmg008') | (data['username'] ==i)]
    #print(fi_data)
    l_all_ka.append(fi_data)
    photo = Digraph('%s' % (i), format='pdf')  # 创建photo图像
    photo.graph_attr['rankdir'] = 'LR' 
    nodes = {}  # 存储所有节点名称的字典

    for index, row in fi_data.iterrows():
            mother = row['proxyName']  # 母公司名称

            if mother in result_yh_dt :
                    #mother_name = str(mother)+ '\n'+str(result_yh_dt[mother]['收款账号'])+'\n'+str(result_yh_dt[mother]['身份证'])
                    mother_name = str(mother)+ '\n'+str(result_yh_dt[mother])
                    #print(mother_name)
            else:
                    mother_name = str(mother)
            child = row['username']
            if child in result_yh_dt :
                #child_name = str(child)+ '\n'+str(result_yh_dt[child]['收款账号'])+'\n'+str(result_yh_dt[child]['身份证'])
                    child_name = str(child)+ '\n'+str(result_yh_dt[child])
            else:
                    child_name = str(child)
    #         else:
    #             s=s+row['用户名 姓名 登录IP']+'\n'
            if mother_name not in nodes.keys():  # 如果没有画过该节点
                photo.node(name=str(mother_name), fontname="Microsoft YaHei")  # 画母公司节点

            if child_name not in nodes.keys():
            # print(str(index))
                photo.node(name=str(child_name), fontname="Microsoft YaHei")  # 画子公司节点

                photo.edge(str(child_name),str(mother_name))
            # print(nodes.items())
            # photo.write_svg(r'D:\代理图\8888.png', prog = 'dot')
    photo.render(filename='%s' % (i),outfile= None,directory=r'D:\项目\大冶\下线会员只有银行卡\{}'.format(i))
    u1=list(fi_data['username'])
    p1=list(fi_data['proxyName'])
    l=list(set(u1+p1))
    data_info=pd.DataFrame({'账号':l})
    data_info['所在代理']=i

    
    data_ka=data_info.merge(df_yh,on='账号',how='left')
    zh.append(data_ka)
    agent_info = data_ka[pd.notnull(data_ka['银行卡开户支行']) & data_ka['银行卡开户支行'].str.contains('湖北省黄石')]
    agent_info['所在代理']=i

    kas.append(agent_info)
    fi_data=fi_data.rename(columns={'proxyName':'所属账户','username':'无方框账户'})
    with pd.ExcelWriter(r'D:\项目\大冶\下线会员只有银行卡\{}\{}.xlsx'.format(i, i)) as writer:
            fi_data.to_excel(writer, sheet_name="关系表",index=False)
            data_ka.to_excel(writer, sheet_name="账号匹配",index=False)
            agent_info.to_excel(writer, sheet_name="黄石账号信息",index=False)
    #data.to_excel(r'C:\Users\23934\Desktop\\代理图1\{}\{}.xlsx'.format(i, i),index=False)
#l_all_ka.to_excel(r'D:\项目\鄂州\代理图\银行卡信息.xlsx',index=False)
agent_kas=pd.concat(l_all_ka)
agent_kas.to_excel(r'D:\项目\大冶\下线会员只有银行卡\关系信息.xlsx',index=False)
zhs=pd.concat(zh)
zhs=zhs.drop_duplicates()
zhs.to_excel(r'D:\项目\大冶\下线会员只有银行卡\银行卡信息.xlsx',index=False)
ka=pd.concat(kas)
ka=ka.drop_duplicates()
ka.to_excel(r'D:\项目\大冶\下线会员只有银行卡\黄石银行卡信息.xlsx',index=False)

C:\Users\23934\AppData\Local\Temp\ipykernel_25540\2245900032.py:120: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  agent_info['所在代理']=i
C:\Users\23934\AppData\Local\Temp\ipykernel_25540\2245900032.py:120: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  agent_info['所在代理']=i
C:\Users\23934\AppData\Local\Temp\ipykernel_25540\2245900032.py:120: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the

In [9]:
import pandas as pd
hyka=pd.read_excel(r"D:\项目\大冶\需要数据\工作簿19.xlsx",sheet_name=0)
hyka.iloc[:,:]=hyka.iloc[:,:].astype("string")
#hyka.to_excel(r'D:\项目\大冶\需要数据\下线会员.xlsx',index=False)

In [10]:
import pandas as pd
hyka_need=pd.read_excel(r"D:\项目\大冶\需要数据\工作簿19.xlsx",sheet_name=1)
hyka_need.iloc[:,:]=hyka_need.iloc[:,:].astype("string")

In [11]:
data_ka=hyka_need.merge(hyka,on='提取银行卡',how='left')

In [12]:
data_ka

,提取银行卡,银行卡号,姓名
0,6013826109004121230,NaN,NaN
1,6013821000628067336,NaN,NaN
2,6229920500113867878,NaN,NaN
3,623059122102865970,NaN,NaN
4,6217002920138976718,NaN,NaN
...,...,...,...
70,6217230406002678758,NaN,NaN
71,6216615007003458427,NaN,NaN
72,6228480246060201661,NaN,NaN
73,6228480389544933374,NaN,NaN
